In [1]:
#TOPSIS-Shannon
import numpy as np               # for linear algebra
import pandas as pd              # for tabular output
from scipy.stats import rankdata # for ranking the candidates
import matplotlib.pyplot as plt

In [2]:
# The given data encoded into vectors and matrices

attributes = np.array(["Security", "QOS", "Cost", "Reputation", "Financial", "Customer", "Performance"])
candidates = np.array(["Amazon","Microsoft","Google","Alibaba","Oracle","IBM"])
raw_data = np.array([
    [4.6, 4.0, 0.120, 4.5, 4.5, 4.5, 4.0],
    [4.5, 4.0, 0.107, 4.4, 4.0, 4.3, 4.0],
    [4.6, 4.0, 0.067, 4.5, 4.0, 4.5, 4.0],
    [4.3, 5.0, 0.198, 4.3, 5.0, 4.4, 4.0],
    [4.4, 5.0, 0.119, 4.3, 3.0, 4.4, 4.0],
    [4.4, 5.0, 0.106, 4.3, 1.0, 4.2, 4.0],
])

weights  = np.array([0.1177, 0.2310, 0.1644, 0.1243, 0.0904, 0.1881, 0.0822])



benefit_attributes = set([0, 1, 0, 3, 4, 5, 6])

# Display the raw data we have
pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
Amazon,4.6,4.0,0.120,4.5,4.5,4.5,4.0
Microsoft,4.5,4.0,0.107,4.4,4.0,4.3,4.0
Google,4.6,4.0,0.067,4.5,4.0,4.5,4.0
Alibaba,4.3,5.0,0.198,4.3,5.0,4.4,4.0
Oracle,4.4,5.0,0.119,4.3,3.0,4.4,4.0
IBM,4.4,5.0,0.106,4.3,1.0,4.2,4.0


In [3]:
m = len(raw_data)
n = len(attributes)
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:,j]
    divisors[j] = np.sqrt(column @ column)

raw_data /= divisors

columns = ["Security", "QOS", "Cost", "Reputation", "Financial", "Customer", "Performance"]
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
Amazon,0.420306,0.360668,0.389456,0.419026,0.481759,0.418990,0.408248
Microsoft,0.411169,0.360668,0.347265,0.409715,0.428230,0.400368,0.408248
Google,0.420306,0.360668,0.217447,0.419026,0.428230,0.418990,0.408248
Alibaba,0.392895,0.450835,0.642603,0.400403,0.535288,0.409679,0.408248
Oracle,0.402032,0.450835,0.386211,0.400403,0.321173,0.409679,0.408248
IBM,0.402032,0.450835,0.344020,0.400403,0.107058,0.391057,0.408248


In [4]:
raw_data *= weights
pd.DataFrame(data=raw_data, index=candidates, columns=columns)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
Amazon,0.049470,0.083314,0.064027,0.052085,0.043551,0.078812,0.033558
Microsoft,0.048395,0.083314,0.057090,0.050928,0.038712,0.075309,0.033558
Google,0.049470,0.083314,0.035748,0.052085,0.038712,0.078812,0.033558
Alibaba,0.046244,0.104143,0.105644,0.049770,0.048390,0.077061,0.033558
Oracle,0.047319,0.104143,0.063493,0.049770,0.029034,0.077061,0.033558
IBM,0.047319,0.104143,0.056557,0.049770,0.009678,0.073558,0.033558


In [5]:
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    
    # See if we want to maximize benefit or minimize cost (for PIS)
    if j in benefit_attributes:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

pd.DataFrame(data=[a_pos, a_neg], index=["$Sj^+$", "$Sj^-$"], columns=columns)

,Security,QOS,Cost,Reputation,Financial,Customer,Performance
$Sj^+$,0.049470,0.104143,0.035748,0.052085,0.048390,0.078812,0.033558
$Sj^-$,0.046244,0.083314,0.105644,0.049770,0.009678,0.073558,0.033558


In [6]:
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = raw_data[i] - a_pos
    diff_neg = raw_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

pd.DataFrame(data=zip(sp, sn, cs), index=candidates, columns=["$di^+$", "$di^-$", "$Ci$"])


,$di^+$,$di^-$,$Ci$
Amazon,0.035453,0.054063,0.603946
Microsoft,0.031587,0.056652,0.642028
Google,0.022967,0.075972,0.767866
Alibaba,0.070030,0.044099,0.386395
Oracle,0.034022,0.050977,0.599735
IBM,0.044376,0.053334,0.545841
